In [23]:
import warnings
import MDAnalysis.analysis.pca
warnings.simplefilter(action='ignore')
import MDAnalysis as mda

In [25]:
path = '/Volumes/fatpidg/IGPS/TRAJECTORIES/tmaritma/'
top = path + 'protein.prmtop'
traj = path + 'tmaritma2_apo.dcd'
u = mda.Universe(top, traj)

In [26]:
from importlib import reload
import mdigest
import mdigest.core.parsetrajectory
import mdigest.core.correlation
reload(mdigest.core.correlation)
reload(mdigest.core.parsetrajectory)
from mdigest.core.parsetrajectory import *
from mdigest.core.correlation import *


In [27]:
# TEST CORRELATION MODULE

In [28]:
mds = MDS()
mds.set_num_replicas(2)
mds.load_system(topology=top, traj_files=traj)
mds.align_traj(inmem=True, selection='name CA')
mds.set_selection('protein and name CA', 'protein')
mds.stride_trajectory(0,1000,1)
# compute correlation from CA displacements for the WT trajectory
dyncorrWT = DynCorr(mds)
dyncorrWT.parse_dynamics(scale=True, normalize=True, LMI='gaussian', MI='None', DCC=True, PCC=True, VERBOSE=True, COV_DISP=True)
# use MI=knn_5_2 to compute generalized based correlation of CA displacements



  0%|          | 0/1000 [00:00<?, ?it/s]

@>: total number of frames in trajectory file: 1000
@>: length of selected trajectory segment:     1000
@>: number of replicas:                     2
@>: using window length of 500 simulation steps
@>: number or frames per replica:           500
@>: first frame:                            0
@>: last frame:                             1000
@>: actual last frame:                      1000
@>: step:                                   1
@>: number of residues in selected atom group: 454
@>: number of nodes    in selected atom group: 454
@>: number of elements in selected atom group: 454
@>: sanity check pass: number of residues is same as number of nodes
@> -- shape of coordinates matrix: (2, 500, 454, 3)
@> -- shape of coordinates_allreplicas matrix: (2, 500, 1362)
@> -- shape of displacements_allreplicas matrix: (2, 500, 1362)
@> -- shape of disp_from_mean_allreplicas matrix: (2, 500, 454)
@> -- shape of distances_allreplicas matrix: (2, 454, 454)
@> -- shape of covar_disp_allreplicas (2,

@>: LMI/MI calculation ...
@>: begin frame: 0
@>: end   frame: 500
@>: step:        1
@>: compute covariance of displacements...
@>: reshaping coordinates...
@>: values array has shape (500, 454, 3)


100%|██████████| 103285/103285 [00:11<00:00, 9176.07it/s]


@>: computing eigenvector centrality from lmi matrix
@>: computing and storing distances...
@>: computing normalized dynamical cross-correlation matrix
@>: computing pearson correlation matrix
@>: LMI/MI calculation ...
@>: begin frame: 500
@>: end   frame: 1000
@>: step:        1
@>: compute covariance of displacements...
@>: reshaping coordinates...
@>: values array has shape (500, 454, 3)


100%|██████████| 103285/103285 [00:11<00:00, 9248.21it/s]


@>: computing eigenvector centrality from lmi matrix
@>: computing and storing distances...
@>: computing normalized dynamical cross-correlation matrix
@>: computing pearson correlation matrix


In [ ]:
# test dim_reduction

In [ ]:
# dimred = DimensionalityReduction()
#
# params = {'universe': u, 'align': 'manual', 'selection': 'backbone',
#           'start':0, 'stop':1000, 'step':1}#, 'align_selection': 'name CA', 'prealign_selection': 'name CA',
# # 'prealign_on_frame': 0}
# dimred.load_data(load_from='universe', z_score=False, **params)
# dimred.dimReduction(method='PCA', n_components=1, mean=None, align=False)
# outpath = '/Users/federica/Desktop/'
# params = {'universe' : u,
#          'selection': 'backbone',
#          'outdir'   : '/Users/federica/Desktop/',
#          'outname'  : 'dimred_projected_traj'}
# # supply mean position and check speeed up sufficient?
#
#
# params = {'universe' : u,
#          'selection': 'backbone',
#          'outdir'   : '/Users/federica/Desktop/',
#          'outname'  : 'dimred_projected_traj'}
# dimred.write_proj_trajectory(**params)

In [116]:
%%time
# from importlib import reload
# import mdigest.core.dimreduction
# reload(mdigest.core.dimreduction)
# from mdigest.core.dimreduction import *
#

import MDAnalysis.analysis.pca as PCA

path = '/Volumes/fatpidg/IGPS/TRAJECTORIES/tmaritma/'
top = path + 'protein.prmtop'
traj = path + 'tmaritma2_apo.dcd'
u = mda.Universe(top, traj)
dimred = PCA.PCA(u, select='backbone', align=True, start=0, stop=1000, step=1).run()


CPU times: user 32min 55s, sys: 2min 49s, total: 35min 44s
Wall time: 2min 45s


In [380]:
u = mda.Universe(top, traj)
mdpca = PCA.PCA(u, select='backbone', align=True, start=0, stop=1000, step=1, mean=None,n_components=1).run()

In [416]:
#mdpca._u.select_atoms('backbone').positions,
print(mdpca._u.trajectory[mdpca.start])
# reference will be start index
print(mdpca._select)
print(mdpca._u.trajectory[mdpca.start].positions[0])
print(mdpca._u.select_atoms(mdpca._select).positions[0])#
print(mdpca._reference.positions[0])#
print('%%%%')
print(mdpca._ref_atom_positions[0])#


new_ref=mda.Merge(mdpca._u.select_atoms(mdpca._select)).load_new(mdpca._ref_atom_positions, order='fac')
print(new_ref.select_atoms('backbone').positions[0])
#print(mdpca._ref_cog)
#print(mdpca._reference.center_of_geometry())

< Timestep 0 with unit cell dimensions [ 81.552826 105.37693   79.72012   90.        90.        90.      ] >
backbone
[-5.723915   3.8945694 15.834141 ]
[-5.723915   3.8945694 15.834141 ]
[-5.723915   3.8945694 15.834141 ]
%%%%
[-6.7557898  2.9853222 14.548609 ]
[-6.7557898  2.9853222 14.548609 ]


In [19]:
%%time
from importlib import reload
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)
simplefilter(action='ignore', category=UserWarning)

import mdigest.core.dimensionalityreduction
reload(mdigest.core.dimensionalityreduction)
from mdigest.core.dimensionalityreduction import *

path = '/Volumes/fatpidg/IGPS/TRAJECTORIES/tmaritma/'
top = path + 'protein.prmtop'
traj = path + 'tmaritma2_apo.dcd'
u = mda.Universe(top, traj)

red = DimensionalityReduction()
params = {'universe': u, 'reference': u, 'selection': 'backbone', 'start':0, 'stop':1000, 'step':1}
red.source_system(**params, align=True)
print(red.mda_u.trajectory[1].positions)

red.MVD(n_components=1, start=params['start'], stop=params['stop'], step=params['step'], align=True)


[[-7.3625374  3.9001923 15.21241  ]
 [-7.6063824  4.19059   16.180527 ]
 [-8.207472   3.6973076 14.7021675]
 ...
 [-5.1336937 32.605804  -1.8303642]
 [-5.4138985 33.41941   -0.9167099]
 [-3.9557836 32.3546    -2.1252937]]
[[-6.587219    3.6607823  16.086956  ]
 [-6.78135     3.9160361  17.075994  ]
 [-7.4578714   3.4879775  15.609957  ]
 ...
 [-4.769124   32.981346    0.07335745]
 [-4.9949813  33.76193     1.029868  ]
 [-3.609902   32.727924   -0.28656316]]
CPU times: user 16 s, sys: 1.54 s, total: 17.5 s
Wall time: 4.62 s


In [20]:
print(red.transformed.shape)
print(red.p_components.shape)
print(red.eigenvectors.shape)
print()

(1000, 1000)
(1, 5448)
(1000, 5448)



In [22]:
outpath = '/Users/federica/Desktop/'
params = {#'universe' : u,
         'selection': 'backbone',
         'outdir'   : '/Users/federica/Desktop/',
         'outname'  : 'projected_traj',
         'start'    : 0,
         'stop'     : 1000,
         'step'     : 1,
         }

universe = red.mda_u.copy()
ref = mda.Merge(universe.select_atoms('backbone'))
red.transformation(ref, components=1)
projected =  red.to_file(**params)

In [367]:
print(red.mda_u.trajectory.timeseries(start=0, stop=-1, step=1).mean(axis=1))

print(dimred.mean)

dimred._ref_atom_positions, red.ref_atom_positions


[[ 4.5394899e-07 -2.1743774e-07  2.9182434e-07]
 [-2.8228760e-07 -6.4468384e-07  5.1879886e-07]
 [-6.8664548e-08  6.2942507e-07 -5.7220461e-07]
 ...
 [-2.2220611e-07  5.6076050e-07 -7.3623659e-07]
 [ 2.3937224e-07 -2.3860932e-06 -4.5490265e-07]
 [-6.7329404e-07 -2.3651123e-07  5.3405763e-08]]
[[-4.5215375   4.08548003 16.63687579]
 [-4.69120163  2.84902952 16.60817499]
 [-4.80563608  2.22351917 15.24579688]
 ...
 [-6.05839188 34.36231888 -3.11493759]
 [-5.11241907 34.73354026 -2.20076686]
 [-4.40435102 34.7070001  -2.07220592]]


(array([[-6.7557898 ,  2.9853222 , 14.548609  ],
        [-6.278014  ,  1.6370671 , 14.821367  ],
        [-6.1126013 ,  0.88402295, 13.559568  ],
        ...,
        [-6.429782  , 32.02885   , -3.2796805 ],
        [-5.398135  , 32.78152   , -2.3286917 ],
        [-5.7611    , 33.51698   , -1.3373511 ]], dtype=float32),
 array([[-5.723915  ,  3.8945694 , 15.834141  ],
        [-5.2461395 ,  2.5463142 , 16.1069    ],
        [-5.0807266 ,  1.7932701 , 14.8451    ],
        ...,
        [-5.3979073 , 32.9381    , -1.9941483 ],
        [-4.3662605 , 33.69077   , -1.0431595 ],
        [-4.729225  , 34.426228  , -0.05181885]], dtype=float32))

In [343]:
%%time
from importlib import reload
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)
simplefilter(action='ignore', category=UserWarning)

import mdigest.core.dimensionalityreduction
reload(mdigest.core.dimensionalityreduction)
from mdigest.core.dimensionalityreduction import *

path = '/Volumes/fatpidg/IGPS/TRAJECTORIES/tmaritma/'
top = path + 'protein.prmtop'
traj = path + 'tmaritma2_apo.dcd'
u = mda.Universe(top, traj)
red2 = DimensionalityReduction()
params = {'universe': u, 'align': 'manual', 'selection': 'backbone', 'start':0, 'stop':1000, 'step':1}
red2.load_data(load_from='universe', z_score=False, **params)
#red.mda_u.trajectory.timeseries(asel=red.mda_u.select_atoms('backbone'), start=0, stop=-1, step=1).mean(axis=1)
red2.MVD(n_components=1, start=params['start'], stop=params['stop'], step=params['step'], align=True)


AttributeError: 'DimensionalityReduction' object has no attribute 'load_data'

In [229]:
red2.cov

array([[  72.54245396,  -31.2014014 , -176.95476097, ...,   68.22686404,
         -21.61697257, -179.62704514],
       [ -31.2014014 ,   14.61742717,   77.06582128, ...,  -29.39325165,
           9.47274852,   78.48167674],
       [-176.95476097,   77.06582128,  438.94745666, ..., -168.3022518 ,
          53.87671801,  444.11164314],
       ...,
       [  68.22686404,  -29.39325165, -168.3022518 , ...,   65.74512456,
         -20.69373929, -170.92996902],
       [ -21.61697257,    9.47274852,   53.87671801, ...,  -20.69373929,
           7.41776358,   54.58996922],
       [-179.62704514,   78.48167674,  444.11164314, ..., -170.92996902,
          54.58996922,  452.02673351]])

In [212]:

coords =red.mda_u.trajectory.timeseries(asel=red.mda_u.select_atoms('backbone'), start=0, stop=-1, step=1).reshape((1000,1816,3))
mean = red.mda_u.trajectory.timeseries(asel=red.mda_u.select_atoms('backbone'), start=1, stop=-1, step=1).mean(axis=1)
coords = np.asarray([coords[i] - mean for i in range(len(coords))]).reshape(1000,-1)
np.cov(coords, rowvar=False)


array([[101.94925621,  22.25426702,   2.26428535, ..., 100.6154295 ,
         22.1067227 ,   2.09774042],
       [ 22.25426702, 283.4369558 ,  -5.40272152, ...,  21.36476708,
        282.08444672,  -5.37490379],
       [  2.26428535,  -5.40272152,  99.46239676, ...,   2.49174879,
         -5.40086387,  98.08333842],
       ...,
       [100.6154295 ,  21.36476708,   2.49174879, ..., 101.81044371,
         21.40029777,   2.11152102],
       [ 22.1067227 , 282.08444672,  -5.40086387, ...,  21.40029777,
        283.55224994,  -5.71225726],
       [  2.09774042,  -5.37490379,  98.08333842, ...,   2.11152102,
         -5.71225726,  99.02817865]])

In [214]:
#mean_centered_data = (red.data - red.mean).reshape(red.data.shape[0], -1)
#red.cov = np.cov(mean_centered_data, rowvar=False)


[0.89975491 1.06236749 1.19616326]
[0.90353993 1.07367368 1.21029264]
[0.8849068  1.07955149 1.20609973]
[0.89660082 1.04889078 1.1847847 ]
[0.86938711 1.07209293 1.18360502]
[0.87290623 1.05418114 1.19006383]
[0.87981653 1.06242609 1.17827761]
[0.8888656  1.05283884 1.18957459]
[0.89122588 1.05778781 1.18392472]
[0.88876327 1.06296674 1.22086794]
[0.89609924 1.07526443 1.20533831]
[0.87188394 1.07718861 1.19950692]
[0.87385749 1.08356371 1.17662731]
[0.89155536 1.08315532 1.20736781]
[0.88888168 1.09407725 1.20563713]
[0.90229589 1.09886483 1.18575405]
[0.8891053  1.08517767 1.19526378]
[0.89446303 1.10602112 1.19291384]
[0.8885422  1.09190571 1.17814805]
[0.8940408  1.09248179 1.17580385]
[0.88630866 1.11479962 1.18323228]
[0.89828512 1.0855039  1.18620747]
[0.8985352  1.09798432 1.16834865]
[0.90264995 1.08702952 1.20279881]
[0.8970494  1.09080666 1.19808552]
[0.88135044 1.09573181 1.20809355]
[0.88418992 1.08320175 1.21601646]
[0.89617935 1.07936847 1.19488364]
[0.88965998 1.089022

In [158]:
tmp = u.copy()
# Define the projection function with the backbone atomgroup

outpath = '/Users/federica/Desktop/'
params = {#'universe' : u,
         'selection': 'backbone',
         'outdir'   : '/Users/federica/Desktop/',
         'outname'  : 'projected_traj',
         'start'    : 0,
         'stop'     : 1000,
         'step'     : 1,
         'components': 0}
red.transformation(components=0)
projected =  red.to_file(**params)

backbone
(1000, 1816, 3)


In [160]:
#red.eigenvectors.T[0]
red.transformed

array([[-1.49368974e-06,  2.44357423e-06,  6.10538725e-08, ...,
        -6.20795858e+02,  2.96820440e+02, -6.46678027e+01],
       [-1.49337003e-06,  2.44350476e-06,  6.10571625e-08, ...,
        -5.60533459e+02,  2.53546927e+02, -3.67692718e+01],
       [-1.49361504e-06,  2.44352429e-06,  6.10781842e-08, ...,
        -4.93609311e+02,  2.77511988e+02,  8.90081332e+00],
       ...,
       [-1.49369603e-06,  2.44358028e-06,  6.10583333e-08, ...,
        -2.77890635e+02, -4.09029171e+02, -1.29138013e+03],
       [-1.49367730e-06,  2.44355213e-06,  6.10948389e-08, ...,
        -2.45865146e+02, -3.91268564e+02, -1.36403586e+03],
       [-1.49363182e-06,  2.44352296e-06,  6.10725646e-08, ...,
        -2.17350447e+02, -2.79856203e+02, -1.38828848e+03]])

In [114]:
coords = np.dot(red.transformed, np.asarray(red.eigenvectors[:,0]).reshape((-1, 1)).T) + red.mean.ravel()

print(coords.shape)


# merged = mda.Merge(u.select_atoms('backbone'))
# u_merged = merged.load_new(coords, order='fac')
# with mda.Writer(outpath + "projected.pdb", u_merged.atoms.n_atoms) as W:
#     W.write(u_merged.atoms)
#     W.close()

(1000, 5448)


(5448, 1)

In [77]:
red.projected


array([[[ 1.2380873 ,  2.42024   ,  1.2589447 ],
        [ 1.2168074 ,  2.3984282 ,  1.2631319 ],
        [ 1.2167556 ,  2.4086928 ,  1.2439597 ],
        ...,
        [ 1.2287453 ,  2.4515254 ,  1.2392229 ],
        [ 1.2189436 ,  2.439918  ,  1.2448201 ],
        [ 1.2246606 ,  2.4450731 ,  1.2311378 ]],

       [[ 1.1019344 ,  1.2779428 ,  1.2901384 ],
        [ 1.0814072 ,  1.258701  ,  1.2943832 ],
        [ 1.0829017 ,  1.266742  ,  1.2749693 ],
        ...,
        [ 1.093188  ,  1.3074561 ,  1.2717837 ],
        [ 1.0833313 ,  1.2972255 ,  1.2773924 ],
        [ 1.0915625 ,  1.3024753 ,  1.2632118 ]],

       [[ 0.9974667 ,  0.40147758,  1.3140726 ],
        [ 0.97751683,  0.38420743,  1.3183618 ],
        [ 0.9801979 ,  0.39054245,  1.2987626 ],
        ...,
        [ 0.9891772 ,  0.42963117,  1.296767  ],
        [ 0.9792783 ,  0.42045695,  1.3023845 ],
        [ 0.9894386 ,  0.42577934,  1.2878217 ]],

       ...,

       [[ 4.6925163 , 31.402252  ,  0.46751088],
        [ 4